In [ ]:
#!pip install git+https://github.com/dgunning/cord19.git

The following part is using search_engine to find papers for analysis

In [ ]:
#fix meta_data_path_problem
def pathfu (dataframe):
    dataframe=dataframe.dropna()
    seqs=dataframe.Path.values
    titles=dataframe.title.values
    allsq=[]
    alltitle=[]
    for seqnu in range(len(seqs)):
        seq=str(seqs[seqnu])
        s1q=seq.split("/")
        sq=[]
        for i in range(7):
            sq.append(s1q[i])
        bases='/'
        bases = bases.join(sq)
        addsq=s1q[7].split(";")
        for i in addsq:
            i=i.replace(' ', '')
            i=i.replace('.json', '')
            base1=bases
            allsq.append(base1+'/'+i+'.json')
            alltitle.append(titles[seqnu])
    returnframe=pd.DataFrame(allsq,columns=['path'])
    returnframe['title']=alltitle
    return returnframe

In [ ]:

import pandas as pd
import numpy as np 
import json
from tqdm import tqdm
import nltk
from nltk.tokenize import sent_tokenize,word_tokenize
from langdetect import detect
#nltk.download('punkt')

In [ ]:
#set up search engigne environment
from cord import ResearchPapers, BIORXIV_MEDRXIV, CUSTOM_LICENSE
from cord.jsonpaper import load_json_cache
research_papers = ResearchPapers.load(data_dir='/home/COVID_NIH/data/',index='text')

In [ ]:
#create total docuemnt title, reading path, and abstract
jasonpath=research_papers.get_json_paths()
title=research_papers.metadata.title
abstract=research_papers.metadata.abstract
cord_uid=research_papers.metadata.cord_uid
allinformation=pd.DataFrame(jasonpath,columns=['Path'])
allinformation['abstract']=abstract
allinformation['title']=title
allinformation['pid']=cord_uid

In [ ]:
searchstr='inhibitor'
fulltextfilename="inhibitor_NIH_full.csv"
fullabstractfilename="inhibitor_NIH_ab.csv"
fulljiffilename="inhibitor_NIH_jif.csv"

In [ ]:
#get COVID title and abstract
covinf=research_papers.covid_related().search(searchst,num_results=1000, covid_related=False,view='table').results[['cord_uid','title','abstract']]
#get NON_COVID title and abstract
notcovinf=research_papers.not_covid_related().search(searchst,num_results=10000, covid_related=False,view='table').results[['cord_uid','title','abstract']]

print(len(covinf))
print(len(notcovinf))

In [ ]:
#get full abstract
fullab=pd.concat([covinf,notcovinf])
fullab=fullab.rename(columns={'cord_uid':'pid'})
fullab=fullab[fullab.abstract!='']
lan=[]
# remove abstract not english
for i in fullab.abstract:
    lan1=detect(i)
    lan.append(lan1)
fullab['lan']=lan
fullab=fullab[fullab.lan=='en']
fullab=fullab[['pid','title','abstract']]
#full_abstract
fullab.to_csv(fullabstractfilename,index=False)



In [ ]:
#read impact factor file and get impact factor for each doc
jif=pd.read_csv('/home/COVID_NIH/data/CORD-19-research-challenge/sjr.csv',sep=';')[['Title','Cites / Doc. (2years)']]
jif=jif.rename(columns={'Title': 'journal', 'Cites / Doc. (2years)': 'IF'})
jif['IF']=jif.IF.str.replace("\"", '')
jif['IF']=jif.IF.str.replace("\," ,'.')
jif['IF']=pd.to_numeric(jif['IF'])


In [ ]:
# covid journal IF
covidjif=research_papers.covid_related().search(searchst,num_results=1000, covid_related=False,view='table').results[['title','journal']]
covidjif=covidjif.merge(jif,how='left')
covidjif['newIF']=np.log(covidjif.IF)+1
covidjif=covidjif[['title','IF']]
# NON_covid journal IF
noncovidjif=research_papers.not_covid_related().search(searchst,num_results=1000, covid_related=False,view='table').results[['title','journal']]
noncovidjif=noncovidjif.merge(jif,how='left')
noncovidjif['newIF']=np.log(noncovidjif.IF)+1
noncovidjif=noncovidjif[['title','IF']]
# full journal IF
fulljif=pd.concat([covidjif,noncovidjif])
fulljif.to_csv(fulljiffilename,index=False)

In [ ]:
#get covid full_text_json_path
covidallinf=allinformation[allinformation.title.isin(covinf.title)]
#get NOn_covid full_text_json_path
noncovidallinf=allinformation[allinformation.title.isin(notcovinf.title)]

In [ ]:
#get covid full_text_json_path
covidallinf=allinformation[allinformation.title.isin(covinf.title)]
#get NOn_covid full_text_json_path
noncovidallinf=allinformation[allinformation.title.isin(notcovinf.title)]

In [ ]:
#get_covid_full_text_frame
returnframe=pathfu(covidallinf)
covid_full_text=fulltextfu(returnframe,json)
#get_NON_covid_full_text_frame
returnframe=pathfu(noncovidallinf)
non_covid_full_text=fulltextfu(returnframe,json)

allframe=pd.concat([covid_full_text,non_covid_full_text])

In [ ]:
allinformation1=allinformation[['title','pid']]
# remove the meta date which title is blank
allinformation1=allinformation1[allinformation1.title!='']
allframe=allframe.merge(allinformation1[['title','pid']])
allframe=allframe[allframe.text!='']
lan=[]
# remove text not english
for i in allframe.text:
    lan1=detect(i)
    lan.append(lan1)
allframe['lan']=lan
allframe=allframe[allframe.lan=='en']
allframe=allframe[['pid','title','text']]
allframe.to_csv(fulltextfilename,index=False)

The following part is using spacy to find releated seed keywords.

In [ ]:
#get each_sentence_entity from document, filter to UMLS, and count the total numbers of the entity in the document 
def entreportfu (dataframe,nlp):
    alldata=pd.DataFrame()
    allent=[]
    for j in dataframe.abstract.values:
        tempnlp=nlp(j)
        tempentlist=list(tempnlp.ents)
        for k in tempentlist:
            if len(k._.umls_ents)>0:
                allent.append(str(k))
    uniqueent=list(set(allent))
    allentnu=[]
    entcategory=[]
    for l in uniqueent:
        entnu=allent.count(l)
        allentnu.append(entnu)
    temp=pd.DataFrame(uniqueent,columns=['entity'])
    temp['entnucount']=allentnu
    temp=temp.sort_values(by='entnucount',ascending=False)
    alldata=pd.concat([alldata,temp])
    return (alldata)

In [ ]:
import spacy
from spacy.matcher import PhraseMatcher
import scispacy
from scispacy.umls_linking import UmlsEntityLinker
nlp = spacy.load('en_core_sci_lg')
linker = UmlsEntityLinker(resolve_abbreviations=True)
nlp.add_pipe(linker)

In [ ]:
covidinfent=entreportfu(covinf,nlp)

In [ ]:
covidinfent.entity.values

In [ ]:
covidinfent[covidinfent.entity.str.contains('virus')]

In [ ]:
#the previous section save to following three files 
#keylist.txt : the classname
#valuelist.txt : the value of each classname
#viruslist.txt the virus list

The following part snorkel_pseudo_list.

In [14]:
import os 
import pandas as pd
from tqdm import tqdm
from snorkel.labeling import PandasLFApplier,LabelModel,LFAnalysis,LabelingFunction
import re
import numpy as np
import nltk
from nltk.tokenize import sent_tokenize,word_tokenize
from sklearn.model_selection import train_test_split
import warnings
import pickle as pkl
import numpy as np
warnings.filterwarnings('ignore')

In [2]:
def build_raw_data (filepath):
    allfile=pd.read_csv(filepath)
    #get alldoucment
    allfile['abstract']=allfile.abstract.astype(str)
    #get allsentence
    allsent=[]
    allid=[]
    for i in tqdm(range(len(allfile))):
        temp=allfile.abstract.iloc[i]
        temp=sent_tokenize(temp)
        for j in range(len(temp)):
            allsent.append(temp[j])
            allid.append(allfile.pid.iloc[i])
            
    allsent=pd.DataFrame(allsent,columns=['sent'])
    allsent['pid']=allid
    
    return allfile, allsent

In [3]:
def loop_labing(keylist,valuelist,virus):
    def keyword_lookup(x, keywords, virus ,label):
        if any(word in x.sent for word in keywords) and any(word in x.sent for word in virus) :
            return label
        return Norelevent

    def make_keyword_lf(keywords, virus,name,label=None):
        return LabelingFunction(
            name=f"keyword_{name}",
            f=keyword_lookup,
            resources=dict(keywords=keywords,virus=virus ,label=label),
        )
    #This function has some drawback because I am writing the function to combine previous and latter sentence for a given sentence
    def abstract_lookup(x, keywords,label):
        if any(word in x.sent for word in keywords):
            return label
        return Norelevent

    def make_abstract_lf(keywords,name,label=None):
        return LabelingFunction(
            name=f"abstract_{name}",
            f=abstract_lookup,
            resources=dict(keywords=keywords,label=label),
        )
    
    Norelevent = -1
    allweaklabf=[]
    viruselist=virus
    for i in range(len(keylist)):
        vbname=keylist[i]
        vbname1=keylist[i]+'ab'
        vbnameab=vbname+'su'
        vbnameab1=vbname+'su1'
        labelvalue=i
        vblabelname=vbname+'l'
     
        globals()[vbname] = make_keyword_lf(keywords=valuelist[i],virus=viruselist,name=vbnameab,label=labelvalue)
        globals()[vbname1] = make_abstract_lf(keywords=valuelist[i],name=vbnameab1,label=labelvalue)
        allweaklabf.append(globals()[vbname])
        allweaklabf.append(globals()[vbname1])
    
    return allweaklabf

In [22]:
def snorkel_process (keylist,dataframe,allweaklabf):
    def func(x):
        idx = (-x).argsort()[1:]
        x[idx] = 0
        return x
    cardinalitynu=len(keylist)
    applier = PandasLFApplier(lfs=allweaklabf)
    all_train_l = applier.apply(df=dataframe)
    report=LFAnalysis(L=all_train_l, lfs=allweaklabf).lf_summary()
    print(report)
    label_model = LabelModel(cardinality=cardinalitynu,verbose=False)
    label_model.fit(all_train_l)
    predt=label_model.predict(all_train_l)
    predt1=label_model.predict_proba(all_train_l)
    keylist1=keylist.copy()
    predt2=pd.DataFrame(predt1,columns=keylist1 )
    dataframe['L_label']=predt
    dataframe1=dataframe.join(predt2, how='outer')
    dataframe1=dataframe1[dataframe1.L_label>=0]
  
    train,test=train_test_split(dataframe1,test_size=0.2)
    
    trainsent=train.sent.values
    
    trainlabel=train[keylist].values
    trainlabe2=trainlabel.copy()
    np.apply_along_axis(func,  1, trainlabe2)
    trainlabe2=np.where(trainlabe2 > 0, 1, 0)
    testsent=test.sent.values
    testlabel=test[keylist].values
    testlabe2=testlabel.copy()
    np.apply_along_axis(func,  1, testlabe2)
    testlabe2=np.where(testlabe2 > 0, 1, 0)
    return trainsent,trainlabel,valsent,vallabel,keylist,report

In [5]:
allfile, allsent=build_raw_data('/home/text_download/inhibitor_NIH_ab.csv')

100%|██████████| 3269/3269 [00:02<00:00, 1393.50it/s]


In [6]:
allsent['newpid']=range(len(allsent))

In [7]:

with open('/home/text_download/keylist.txt', "r") as f:
    alist =f.read().splitlines()
    for line in alist:
        keylist=line.split(',')
    

In [8]:
valuelist = []
with open('/home/text_download/valuelist.txt', "r") as f:
    alist =f.read().splitlines()
    #alist =[x.lower() for x in alist]
    for line in alist:
        valuelist.append(line.split(','))

In [9]:

with open('/home/text_download/viruslist.txt', "r") as f:
    alist =f.read().splitlines()
    for line in alist:
        viruslist=line.split(',')
        #viruslist =[x.lower() for x in viruslist]

In [10]:
allweaklabf=loop_labing(keylist,valuelist,viruslist)

In [23]:
trainsent,trainlabel,valsent,vallabel,keylist,report=snorkel_process (keylist,allsent,allweaklabf)

100%|██████████| 25739/25739 [00:42<00:00, 610.54it/s]


                               j Polarity  Coverage  Overlaps  Conflicts
keyword_antihivsu              0      [0]  0.000272  0.000272   0.000000
abstract_antihivsu1            1      [0]  0.001476  0.000505   0.000233
keyword_antihcvsu              2      [1]  0.000117  0.000117   0.000000
abstract_antihcvsu1            3      [1]  0.000311  0.000117   0.000000
keyword_anti-parasitesu        4       []  0.000000  0.000000   0.000000
abstract_anti-parasitesu1      5      [2]  0.000466  0.000039   0.000039
keyword_immue_suppresivesu     6       []  0.000000  0.000000   0.000000
abstract_immue_suppresivesu1   7      [3]  0.000427  0.000155   0.000155
keyword_antivirussu            8      [4]  0.000350  0.000350   0.000000
abstract_antivirussu1          9      [4]  0.003924  0.000544   0.000194
keyword_interleukinsu         10      [5]  0.000155  0.000155   0.000000
abstract_interleukinsu1       11      [5]  0.007693  0.000505   0.000350
keyword_cancerdrugsu          12       []  0.000000

In [27]:
pd.DataFrame(trainsent,columns=['sent']).to_csv("trainsent.csv")
pd.DataFrame(trainlabel,columns=keylist).to_csv("trainlabel.csv" )
pd.DataFrame(valsent,columns=['sent']).to_csv("valsent.csv")
pd.DataFrame(vallabel,columns=keylist).to_csv("vallabel.csv")
    